In [1]:
import os
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
from api_keys import Client_ID, Client_Secret, Username, redirect_uri
import sys
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import json
from pprint import pprint

creating the tabular data

In [89]:
scope = 'user-library-read'

token = util.prompt_for_user_token(username=Username,
                                   scope=scope,
                                   client_id=Client_ID,
                                   client_secret=Client_Secret,
                                   redirect_uri=redirect_uri)


sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, auth=token)


# pprint(sp.current_user_playlists()['items'][0])

playlist_one = sp.current_user_playlists()['items'][4]
print('----------------')
# pprint(playlist_one)

playlist_one_id = playlist_one['id']
print('----------------')
# pprint(sp.playlist_tracks(playlist_one_id)['items'])
playlist_tracks = sp.playlist_tracks(playlist_one_id)['items'][0]['track']
pprint(playlist_tracks)

----------------
----------------
{'album': {'album_type': 'album',
           'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7C3Cbtr2PkH2l4tOGhtCsk'},
                        'href': 'https://api.spotify.com/v1/artists/7C3Cbtr2PkH2l4tOGhtCsk',
                        'id': '7C3Cbtr2PkH2l4tOGhtCsk',
                        'name': 'Shallou',
                        'type': 'artist',
                        'uri': 'spotify:artist:7C3Cbtr2PkH2l4tOGhtCsk'}],
           'available_markets': ['AD',
                                 'AE',
                                 'AR',
                                 'AT',
                                 'AU',
                                 'BE',
                                 'BG',
                                 'BH',
                                 'BO',
                                 'BR',
                                 'CA',
                                 'CH',
                                 'CL',
     

In [86]:
for i in range(10):
    playlist_name = sp.current_user_playlists()['items'][i]['name']
    print(playlist_name)
    playlist_id = sp.current_user_playlists()['items'][i]['id']
    print(playlist_id)

Starred
686RTHP64rqidDpEnVEGp1
Relax & Unwind
37i9dQZF1DWU0ScTcjJBdj
mint
37i9dQZF1DX4dyzvuaRJ0n
life is strange
0uYqF7OG2e4sTTE6daYfR8
Brain Food
37i9dQZF1DWXLeA8Omikj7
Your Coffee Break
37i9dQZF1DWYqdkUCLfYzP
Burnout Revenge Soundtracks
3rSydatbzsWdosegb5HlcB
Daft Punk — Discovery
4kBiRHYroh8v3eQ6KkSZtG
Stuff
4dVHQlNZMAOkCKXu6qkLqu
Motivational
44VCWYogkSiZ9FDcPuMuvh


In [19]:
data_df = pd.read_csv(os.path.join('data','data.csv'))
data_by_artist_df = pd.read_csv(os.path.join('data','data_by_artist.csv'))
data_by_genres_df = pd.read_csv(os.path.join('data','data_by_genres.csv'))
data_by_year_df = pd.read_csv(os.path.join('data','data_by_year.csv'))
data_w_genres_df = pd.read_csv(os.path.join('data','data_w_genres.csv'))

In [6]:
# This whole thing below is from [https://medium.com/@samlupton/spotipy-get-features-from-your-favourite-songs-in-python-6d71f0172df0]

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm.notebook import tqdm

client_credentials_manager = SpotifyClientCredentials(client_id=Client_ID, client_secret=Client_Secret)

scope = 'user-library-read'

token = util.prompt_for_user_token(username=Username,
                                   scope=scope,
                                   client_id=Client_ID,
                                   client_secret=Client_Secret,
                                   redirect_uri=redirect_uri)

# cached_token = client_credentials_manager.get_access_token()

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, auth=token)

scope = 'user-library-read'


def artist_tracks(artists):
    
    '''
    Takes a list of artist names, iterates through their Spotify albums, checks for 
    duplicate albums, then appends all the tracks in those albums to a list of lists
    '''
    
    # Each list in this list will be a track and its features
    tracks = []
    
    
    
    for artist in tqdm(artists):
        
        # Get the artist URI (a unique ID)
        artist_uri = sp.search(artist)['tracks']['items'][0]['artists'][0]['uri']

        # Spotify has a lot of duplicate albums, but we'll cross-reference them with this list to avoid extra loops
        album_checker = []
        
        # The starting point of our loop of albums for those artists with more than 50
        n = 0
        
        # Note the album_type = 'album'. This discounts singles, compilations and collaborations
        while len(sp.artist_albums(artist_uri, album_type = 'album', limit=50, offset = n)['items']) > 0:
            
            # Avoid overloading Spotify with requests by assigning the list of album dictionaries to a variable
            dict_list = sp.artist_albums(artist_uri, album_type = 'album', limit=50, offset = n)['items']
            
            for i, album in tqdm(enumerate(dict_list)):

                # Add the featured artists for the album in question to the checklist
                check_this_album = [j['name'] for j in dict_list[i]['artists']]
                # And the album name
                check_this_album.append(dict_list[i]['name'])
                # And its date
                check_this_album.append(dict_list[i]['release_date'])

                # Only continue looping if that album isn't in the checklist
                if check_this_album not in album_checker:
                    
                    # Add this album to the checker
                    album_checker.append(check_this_album)
                    # For every song on the album, get its descriptors and features in a list and add to the tracklist
                    tracks.extend([[artist, album['name'], album['uri'], song['name'],

                      album['release_date']] + list(sp.audio_features(song['uri'])[0].values()) 
                                   for song in sp.album_tracks(album['uri'])['items']])
            
            # Go through the next 50 albums (otherwise we'll get an infinite while loop)
            n += 50

    return tracks

artist_tracks(["My Chemical Romance"])

[['My Chemical Romance',
  'The Black Parade / Living with Ghosts (The 10th Anniversary Edition)',
  'spotify:album:29mlJg2PpuihKDNYZalIxw',
  'The End.',
  '2016-09-23',
  0.292,
  0.548,
  2,
  -4.977,
  1,
  0.0363,
  0.0827,
  0,
  0.414,
  0.207,
  175.053,
  'audio_features',
  '7oiqxhRXTv11kRqEV0D70G',
  'spotify:track:7oiqxhRXTv11kRqEV0D70G',
  'https://api.spotify.com/v1/tracks/7oiqxhRXTv11kRqEV0D70G',
  'https://api.spotify.com/v1/audio-analysis/7oiqxhRXTv11kRqEV0D70G',
  112920,
  3],
 ['My Chemical Romance',
  'The Black Parade / Living with Ghosts (The 10th Anniversary Edition)',
  'spotify:album:29mlJg2PpuihKDNYZalIxw',
  'Dead!',
  '2016-09-23',
  0.481,
  0.973,
  7,
  -1.945,
  1,
  0.0542,
  0.0037,
  0.000877,
  0.325,
  0.362,
  99.331,
  'audio_features',
  '3262RRKqFC1oJXFwCMqgM0',
  'spotify:track:3262RRKqFC1oJXFwCMqgM0',
  'https://api.spotify.com/v1/tracks/3262RRKqFC1oJXFwCMqgM0',
  'https://api.spotify.com/v1/audio-analysis/3262RRKqFC1oJXFwCMqgM0',
  195520,
 

In [5]:
worlds_longest_playlist_id = '6yPiKpy7evrwvZodByKvM9'
metropolis_playlist_id = '37i9dQZF1DX8CopunbDxgW'
vapourwave_playlist_id = '7AfWDGHu83rrBqvpg9reh9'